<a href="https://colab.research.google.com/github/BarCodeReader/8800scripts/blob/master/Colab's_New_Code_Editor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.nn.utils import weight_norm
from torch.utils.data import Dataset
from torchsummary import summary

import glob
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
from tqdm import tqdm_notebook as tqnote

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
device

In [0]:
from google.colab import drive
drive.mount('/gdrive')

In [0]:
files = glob.glob('/gdrive/My Drive/Kannada-MNIST/*csv')
print(files)

In [0]:
train = pd.read_csv('/gdrive/My Drive/Kannada-MNIST/train.csv')
test = pd.read_csv('/gdrive/My Drive/Kannada-MNIST/test.csv')
val = pd.read_csv('/gdrive/My Drive/Kannada-MNIST/Dig-MNIST.csv')

In [0]:
test.head(5)

In [0]:
GT_train = np.array(train['label'])
GT_val = np.array(val['label'])
test_idx = np.array(test['id'])

In [0]:
train_len = GT_train.shape[0]
val_len = GT_val.shape[0]
test_len = test_idx.shape[0]

In [0]:
train_data = np.array([np.array(train.iloc[i,1:]).reshape((28,28)) for i in range(train_len)])
val_data = np.array([np.array(train.iloc[k,1:]).reshape((28,28)) for k in range(val_len)])
test_data = np.array([np.array(test.iloc[j,1:]).reshape((28,28)) for j in range(test_len)])

In [0]:
print(train_data.shape)
print(val_data.shape)
print(test_data.shape)

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
x_train, x_dev, y_train, y_dev = train_test_split(train_data,GT_train,test_size=0.1, random_state=42)

In [0]:
x_train = x_train/255.
x_dev = x_dev/255.

val_data = val_data/255.

In [0]:
class DataGen(Dataset):
  def __init__(self,imgset,GT,
               shuffle=False,drop_last=True,transform = None):
    
    self.np_file = len(imgset)
    self.shuffle=shuffle
    
    self.imgset = imgset
    self.GT = GT
        
  def __len__(self):
    return self.np_file

  def __getitem__(self,idx):
    
    seed = np.random.randint(0,self.np_file)
    
    img1, img2, img3, img4 = self.transform(self.imgset[seed])
    truth = self.GT[seed]
    
    #reshape
    img1 = img1.reshape((1,28,28))
    img2 = img2.reshape((1,28,28))
    img3 = img3.reshape((1,28,28))
    img4 = img4.reshape((1,28,28))
    
    img1 = torch.from_numpy(img1).float()
    img2 = torch.from_numpy(img2).float()
    img3 = torch.from_numpy(img3).float()
    img4 = torch.from_numpy(img4).float()
    
    truth = torch.torch.tensor(truth)#.float()
    return img1, img2, img3, img4, truth
  
  def transform(self,info):
    '''     
       graph 1      graph 2      graph 3      graph 4
      ________     ________     ________     ________
      | 1 | 2 |    | 2 | 1 |    | 3 | 4 |    | 4 | 3 |
      |___|___|    |___|___|    |___|___|    |___|___|
      | 3 | 4 |    | 4 | 3 |    | 1 | 2 |    | 2 | 1 |
      |___|___|    |___|___|    |___|___|    |___|___|  

    '''
    #divide into parts
    part1 = info[:14,:14]
    part2 = info[:14,14:]
    part3 = info[14:,:14]
    part4 = info[14:,14:]

    #graph 2
    g2_upper = np.concatenate((part2,part1),axis = 1)
    g2_lower = np.concatenate((part4,part3),axis = 1)
    img2 = np.concatenate((g2_upper,g2_lower),axis = 0)

    #graph 3
    g3_upper = np.concatenate((part3,part4),axis = 1)
    g3_lower = np.concatenate((part1,part2),axis = 1)
    img3 = np.concatenate((g3_upper,g3_lower),axis = 0)

    #graph 4
    g4_upper = np.concatenate((part4,part3),axis = 1)
    g4_lower = np.concatenate((part2,part1),axis = 1)
    img4 = np.concatenate((g4_upper,g4_lower),axis = 0)
  
    return info, img2, img3, img4

In [0]:
pic = DataGen(val_data,GT_val)

In [0]:
p1,p2,p3,p4,gt = next(iter(pic))

In [0]:
p1.size()

In [0]:
plt.figure(figsize=[20,5])
plt.subplot(1,4,1)
plt.imshow(p1.numpy()[0])

plt.subplot(1,4,2)
plt.imshow(p2.numpy()[0])

plt.subplot(1,4,3)
plt.imshow(p3.numpy()[0])

plt.subplot(1,4,4)
plt.imshow(p4.numpy()[0])


In [0]:
gt

In [0]:
#define the model

class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        #for channel 1
        self.c1conv1 = nn.Conv2d(1,32,kernel_size = (3,3))
        self.c1conv2 = nn.Conv2d(32,32,kernel_size = (3,3))
        self.c1conv3 = nn.Conv2d(32,64,kernel_size = (3,3))
        self.c1conv4 = nn.Conv2d(64,64,kernel_size = (3,3))

        #for channel 2
        self.c2conv1 = nn.Conv2d(1,32,kernel_size = (3,3))
        self.c2conv2 = nn.Conv2d(32,32,kernel_size = (3,3))
        self.c2conv3 = nn.Conv2d(32,64,kernel_size = (3,3))
        self.c2conv4 = nn.Conv2d(64,64,kernel_size = (3,3))
        
        #for channel 3
        self.c3conv1 = nn.Conv2d(1,32,kernel_size = (3,3))
        self.c3conv2 = nn.Conv2d(32,32,kernel_size = (3,3))
        self.c3conv3 = nn.Conv2d(32,64,kernel_size = (3,3))
        self.c3conv4 = nn.Conv2d(64,64,kernel_size = (3,3))
        
        #for channel 4
        self.c4conv1 = nn.Conv2d(1,32,kernel_size = (3,3))
        self.c4conv2 = nn.Conv2d(32,32,kernel_size = (3,3))
        self.c4conv3 = nn.Conv2d(32,64,kernel_size = (3,3))
        self.c4conv4 = nn.Conv2d(64,64,kernel_size = (3,3))
        
        self.flatten = nn.Flatten()
        self.pool = nn.MaxPool2d((2,2))
        self.dropout = nn.Dropout()
        
        #for channel 1
        self.c1fc1 = nn.Linear(1024,128)
        self.c1fc2 = nn.Linear(128,128)
        self.c1fc3 = nn.Linear(128,10)    

        #for channel 2
        self.c2fc1 = nn.Linear(1024,128)
        self.c2fc2 = nn.Linear(128,128)
        self.c2fc3 = nn.Linear(128,10) 
        
        #for channel 3
        self.c3fc1 = nn.Linear(1024,128)
        self.c3fc2 = nn.Linear(128,128)
        self.c3fc3 = nn.Linear(128,10) 
        
        #for channel 4
        self.c4fc1 = nn.Linear(1024,128)
        self.c4fc2 = nn.Linear(128,128)
        self.c4fc3 = nn.Linear(128,10) 
        
    def forward(self, c1,c2,c3,c4):
      
      #for channel 1
      x1 = F.relu(self.c1conv1(c1))
      x1 = F.relu(self.c1conv2(x1))
      
      x1 = self.pool(x1)
      
      x1 = F.relu(self.c1conv3(x1))
      x1 = F.relu(self.c1conv4(x1))      
      x1 = self.pool(x1)
      
      x1 = self.flatten(x1)
      x1 = F.relu(self.c1fc1(x1))
      x1 = self.dropout(x1)
      
      x1 = F.relu(self.c1fc2(x1))
      x1 = self.dropout(x1)     
      
      x1 = F.softmax(self.c1fc3(x1))

      #for channel 2
      x2 = F.relu(self.c2conv1(c2))
      x2 = F.relu(self.c2conv2(x2))
      
      x2 = self.pool(x2)
      
      x2 = F.relu(self.c2conv3(x2))
      x2 = F.relu(self.c2conv4(x2))      
      x2 = self.pool(x2)
      
      x2 = self.flatten(x2)
      x2 = F.relu(self.c2fc1(x2))
      x2 = self.dropout(x2)
      
      x2 = F.relu(self.c2fc2(x2))
      x2 = self.dropout(x2)     
      
      x2 = F.softmax(self.c2fc3(x2))
      
      #for channel 3
      x3 = F.relu(self.c3conv1(c3))
      x3 = F.relu(self.c3conv2(x3))
      
      x3 = self.pool(x3)
      
      x3 = F.relu(self.c3conv3(x3))
      x3 = F.relu(self.c3conv4(x3))      
      x3 = self.pool(x3)
      
      x3 = self.flatten(x3)
      x3 = F.relu(self.c3fc1(x3))
      x3 = self.dropout(x3)
      
      x3 = F.relu(self.c3fc2(x3))
      x3 = self.dropout(x3)     
      
      x3 = F.softmax(self.c3fc3(x3))
      
      #for channel 4
      x4 = F.relu(self.c4conv1(c4))
      x4 = F.relu(self.c4conv2(x4))
      
      x4 = self.pool(x4)
      
      x4 = F.relu(self.c4conv3(x4))
      x4 = F.relu(self.c4conv4(x4))      
      x4 = self.pool(x4)
      
      x4 = self.flatten(x4)
      x4 = F.relu(self.c4fc1(x4))
      x4 = self.dropout(x4)
      
      x4 = F.relu(self.c4fc2(x4))
      x4 = self.dropout(x4)     
      
      x4 = F.softmax(self.c4fc3(x4))
      
      out = (x1+x2+x3+x4)/4
      
      return out

In [0]:
def worker_init_fn(worker_id):
    np.random.seed(np.random.get_state()[1][0] + worker_id)

In [0]:
def fit_on_generator(model, optimizer, loss_func, training,
                     validation = None, 
                     steps_train = 100, steps_val = 100, checkpoint = None, verbose = 1):
  
    history_train = []
    history_val = []
    train_loss = np.zeros((steps_train))

    model.train()
    count = 0
    with tqnote(total=steps_train) as pbar:
        for x1,x2,x3,x4,GT in training:

            x1 = x1.to(device)
            x2 = x2.to(device)
            x3 = x3.to(device)
            x4 = x4.to(device)
            
            optimizer.zero_grad()
            y_pred = model(x1,x2,x3,x4)

            loss = loss_func(y_pred,GT)
            loss.backward()  
            
            train_loss[count] = loss.cpu().data.numpy()
            optimizer.step()
            count += 1
            pbar.update(1)
            if count == steps_train:
                break
    history_train.append(np.mean(train_loss))
    if verbose:
        print('Training loss: {}'.format(history_train[-1]))

    #--------------------------------------------------
    if validation is not None:
        print('now on validation...')
        val_loss = np.zeros((steps_val))
        count = 0
        model.eval()
        for xv1,xv2,xv3,xv4,GTv in validation:

            xv1 = xv1.to(device)
            xv2 = xv2.to(device)
            xv3 = xv3.to(device)
            xv4 = xv4.to(device)
            
            y_predv = model(xv1,xv2,xv3,xv4)

            lossv = loss_func(y_predv,GTv)
            val_loss[count] = lossv.cpu().data.numpy()
            count += 1
                
            pbar.update(1)
            if count == steps_val:
                break

        history_val.append(np.mean(val_loss))
        
        print('Validation loss: {}'.format(history_val[-1]))
        if checkpoint is not None:
            checkpoint(history_val[-1], model)
    return {'train_loss':history_train, 'val_loss':history_val}

In [0]:
train = DataGen(x_train,y_train)
dev = DataGen(x_dev,y_dev)
val = DataGen(val_data,GT_val)

In [0]:
bs = 10

train_data = DataLoader(train, batch_size=bs,
                  shuffle=True, num_workers=8,
                  drop_last=True, worker_init_fn = worker_init_fn)
        
val_data = DataLoader(val, batch_size=bs,
                 shuffle=True, num_workers=8,
                 drop_last=True, worker_init_fn = worker_init_fn)

In [0]:
#load model
model = CNN()
model.to(device)
print(model)

In [0]:
loss = torch.nn.CrossEntropyLoss()
loss.to(device)

optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)

In [0]:
epochs = 100
n_train = 50# len(train_data)
n_val = len(val_data)

for epoch in range(epochs):
    np.random.seed()
    print('Epoch {}/{}'.format(epoch, epochs))
    history = fit_on_generator(model, optimizer, loss,  
                               training = train_data,
                               validation = val_data,
                               steps_train = n_train,
                               steps_val = n_val,
                               verbose = 1)